# Working with Config Files

Python provides a convinient way to work with configuration files similar to Windows INI files. First you create a config file, and in general with the extension `.cfg`, like `my_config.cfg`. For this example we have a file called `my_favorites.cfg` divide in sections.

```
[FOOD]
favorite = pizza
weekly = yes
how_often_wk = 1
# The price in SAR.
price = 45.5
```

## Reading Config File

To read a config file, the first thing to do is to import the `configparser` module

In [1]:
import configparser as CP

Next we read the file, and set the _interpolation_ to use. I recommend to always use the _ExtendedInterpolation_: this allows you to access values in other sections and provide a nicer way to do it.

In [2]:
my_config = CP.ConfigParser(interpolation=CP.ExtendedInterpolation())
my_config.read("my_favorites.cfg")

['my_favorites.cfg']

## Sections

The config file can be divided in _sections_ so we can group the values in a logical way. You can check if a section of the document exists before trying to read it, and causing the program to crash.

In [3]:
if my_config.has_section("GAME"):
    my_fav_game = my_config["GAME"]["favorite"]
    print(f"My favorite game is '{my_fav_game}'.")
else:
    print("No section GAME defined")
    # You can get a list of all sections available
    print(f"We only have {', '.join(ss for ss in my_config.sections())}")
    my_fav_game = ""

No section GAME defined
We only have FOOD, DRINK


## Values

Inside the section, we have the actual values that we are going to use. Note that Python doesn't try to gess the datatypes of the values, and stored them internally as _string_. But since it's very common to read strings, integers, booleans, etc., values, Python offers some convinent functions to return the correct datatype.

We can define a function that read all values and returns a dictionary. If we don't care about the datatypes, the function can be compact

In [4]:
def read_config(config):
    """
    Read a section of a ConfigParse and returns dictionary.
    Note: all values will be 'string', and you will have to 
    cast to the correct type later syourself.
    """
    my_dict = {}

    for kk, vv in config.items():
        my_dict.update({kk: vv})
    return my_dict

In [5]:
my_food = read_config(my_config["FOOD"])
my_drinks = read_config(my_config["DRINK"])

In [7]:
print(my_food)
type(my_food['price'])

{'favorite': 'pizza', 'weekly': 'yes', 'how_often_wk': '1', 'price': '45.5'}


str

As you can see, the type of `price` is `str`, that is "string." But most probably we will use a float value for computations, so we will need to convert first:

In [8]:
# Convert to float:
food_price = float(my_food["price"])
drink_price = float(my_drinks["price"])

print(f"The price of a meal is {food_price + drink_price} SAR")

The price of a meal is 58.7 SAR


We can create a function to read the values of the section with the correct datatypes, but then it will be less generic.

In [23]:
def read_food(config):
    """
    Read the 'FOOD' section respecting the type of the variables.
    """

    my_dict = {}

    my_dict.update({"food": config.get("favorite", "lasgna")})
    my_dict.update({"weekly": config.getboolean("weekly", True)})
    my_dict.update({"times": config.getint("how_often_wk", 5)})
    my_dict.update({"price": config.getfloat("price", 50.0)})
    my_dict.update({"healthy": config.getboolean("healthy", False)})

    return my_dict

> Notice that we added an interesting feature to our function: a _fallback_ value that will be used if it's missing from the config file.

So if we want to repeat the previous example, then we won't need to convert the value

In [24]:
# We read the values using the second function
my_food = read_food(my_config["FOOD"])
# And no convertion this time
type(my_food['price'])

float

This can be especially handy for boolean values since the string "False" will be treated as "true" because it has a value instead of empty. Consider the case for the favorite food, the value for `healthy` (if it's good, can't be healthy!) is _false_, but Python thinks otherwise:

In [28]:
if my_food['healthy']:
    print(f"You are luck that {my_food['favorite']} is also healthy.")
else:
    print(f"Never mind, just don't eat to much {my_food['food']}.") 

Never mind, just don't eat to much pizza.
